In [1]:
import os
import json
import ast
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [2]:
IMG_DIR = Path("sandbox/train_sample")  

MODEL_PATH = Path("resnet50_NLP_attributes_final.pth")

OUTPUT_CSV = Path("predicted_attributes_final.csv")

NUM_ATTRIBUTES = 294

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [3]:
class ImageOnlyDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = Path(img_dir)
        self.img_files = list(self.img_dir.glob("*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_path.stem

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
dataset = ImageOnlyDataset(IMG_DIR, transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [5]:
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, NUM_ATTRIBUTES)

print(f"Loading model from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

/Users/kashyapj/Desktop/NLP_Project/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kashyapj/Desktop/NLP_Project/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading model from: resnet50_NLP_attributes_final.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
results = []
with torch.no_grad():
    for imgs, img_ids in tqdm(loader, desc="Predicting attributes"):
        imgs = imgs.to(device)
        outputs = model(imgs) 
        probs = torch.sigmoid(outputs)    
        preds = (probs > 0.5).int().cpu().numpy()  

        for image_id, pred_vector in zip(img_ids, preds):
            results.append({
                "image_id": image_id,
                "attributes": list(pred_vector)
            })

df_preds = pd.DataFrame(results)
df_preds.to_csv(OUTPUT_CSV, index=False)
print(f"Predicted attributes saved to {OUTPUT_CSV}")

Predicting attributes: 100%|██████████| 125/125 [12:33<00:00,  6.03s/it]


Predicted attributes saved to predicted_attributes_final.csv


In [ ]:
import re
import ast
import pandas as pd
import numpy as np
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report,

)


def parse_np_int_list(s):
   
    digits = re.findall(r"np\.int32\(\s*(\d+)\s*\)", s)
    if digits:
        return [int(x) for x in digits]
   
    return ast.literal_eval(s)

def parse_float_list(s):
    
    return ast.literal_eval(s)


gt_df = pd.read_csv(
    "sandbox/train_attribute_data.csv",
    converters={"attributes": parse_np_int_list}
)


pred_df = pd.read_csv(
    "predicted_attributes_final.csv",
    converters={
        "attributes": parse_np_int_list,
       
    }
)

merged = pd.merge(
    gt_df.rename(columns={"attributes":"attributes_true"}),
    pred_df.rename(columns={"attributes":"attributes_pred"}),
    on="image_id"
)

y_true = np.vstack(merged["attributes_true"].values)  
y_pred = np.vstack(merged["attributes_pred"].values)


print("Shapes:", y_true.shape, y_pred.shape)
print("Unique true labels:", np.unique(y_true))
print("Unique pred labels:", np.unique(y_pred))
macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)
macro_p  = precision_score(y_true, y_pred, average="macro", zero_division=0)
macro_r  = recall_score   (y_true, y_pred, average="macro", zero_division=0)

print(f"Macro Precision: {macro_p:.4f}")
print(f"Macro Recall:    {macro_r:.4f}")
print(f"Macro F1:        {macro_f1:.4f}")
print(f"Micro F1:        {micro_f1:.4f}\n")

print("Per-attribute classification report:")
print(classification_report(
    y_true,
    y_pred,
    zero_division=0,
    target_names=[f"attr_{i}" for i in range(y_true.shape[1])]
))





Shapes: (2000, 294) (2000, 294)
Unique true labels: [0 1]
Unique pred labels: [0 1]
Macro Precision: 0.2605
Macro Recall:    0.1511
Macro F1:        0.1731
Micro F1:        0.6484

Per-attribute classification report:
              precision    recall  f1-score   support

      attr_0       0.84      0.82      0.83       239
      attr_1       0.00      0.00      0.00         8
      attr_2       0.00      0.00      0.00        27
      attr_3       0.00      0.00      0.00         4
      attr_4       0.00      0.00      0.00         3
      attr_5       0.00      0.00      0.00         6
      attr_6       0.00      0.00      0.00         2
      attr_7       0.00      0.00      0.00        10
      attr_8       1.00      0.39      0.56        49
      attr_9       0.00      0.00      0.00        16
     attr_10       0.90      0.31      0.46        61
     attr_11       0.94      0.70      0.80        92
     attr_12       0.00      0.00      0.00        10
     attr_13       0.00  

In [ ]:
import pandas as pd
import numpy as np
import ast
import json

GT_CSV      = "predicted_attributes_final.csv"
LABELS_JSON = "sandbox/label_descriptions.json"
def parse_np_int_list(s):
       
    digits = re.findall(r"np\.int32\(\s*(\d+)\s*\)", s)
    if digits:
        return [int(x) for x in digits]
   
    return ast.literal_eval(s)

def parse_float_list(s):
    
    return ast.literal_eval(s)

gt_df = pd.read_csv(
    "predicted_attributes_final.csv",
    converters={
        "attributes": parse_np_int_list,
       
    }
)


with open(LABELS_JSON) as f:
    label_data = json.load(f)
attr_names = [a["name"] for a in label_data["attributes"]]

counts = np.vstack(gt["attributes"].values).sum(axis=0)
df_freq = pd.DataFrame({
    "attr_id":   np.arange(len(counts)),
    "name":      attr_names,
    "frequency": counts
})

df_rare = df_freq.sort_values("frequency").reset_index(drop=True)

K = 10
print(f"Top {K} rarest attributes:")
print(df_rare.head(K).to_string(index=False))


Top 10 rarest attributes:
 attr_id           name  frequency
      87       raincoat          0
     246      snakeskin          0
      99     cheongsams          0
      53 cargo (shorts)          0
      55    boardshorts          0
     238        feather          0
      25   mao (jacket)          0
      66  cargo (skirt)          0
      41       culottes          0
     240           bone          0
